## 학습목표
 1. dataframe 제공 함수를 통한 데이터 처리 기법 숙지
 2. groupby, concat, merge, pivot, pivot_table, stack, unstack, transorm 등

### group by
  + 아래의 세 단계를 포함 (SQL의 group by 와 개념적으로는 동일, 사용법은 유사)
    - 데이터 분할
    - operation 적용
    - 데이터 병합

# a['a'] is a[0] why False?

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
a = pd.Series(range(4), index=['a', 'b', 'c', 'd'])

In [ ]:
# 1955년부터의 유로리그 우승팀 데이터
df = pd.read_csv('./data/euro_winners.csv')

In [ ]:
df.head(15)

In [ ]:
# 국가별로 그룹핑 
nation_group = df.groupby('Nation')
type(nation_group)

In [ ]:
nation_group.groups # groups 속성으로 group 정보를 dict로 반환

In [ ]:
len(nation_group.groups) # 그룹의 개수

* group 함수
 - count, size 등

In [ ]:
nation_group.count()

In [ ]:
nation_group.size()

In [ ]:
nation_group.size().sort_values(ascending = False)

In [ ]:
# 2개 이상의 컬럼으로 grouping 가능
club_group = df.groupby(['Nation', 'Winners'])
club_group.size().sort_values(ascending = False)

* 각 리그별 경기수, 득점 데이터

In [ ]:
df = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [ ]:
df.head(20)

* inplace parameter
 - 대부분의 dataframe 연산은 원본데이터를 유지하고 복사본을 반환
 - 원본 자체를 변경하고 싶으면 inplace paramter를 True로 설정

In [ ]:
df.head()

In [ ]:
# Month column을 index로 생성
df.set_index('Month', inplace=True)

# 혹은 아래와 같이 가능
#df.set_index(['Month', 'Stat'], inplace=True)

In [ ]:
df.head()

In [ ]:
# 다시 index를 column 레벨로 변경
df = df.reset_index()

In [ ]:
df.head(100)

* groupby에 함수 사용

In [ ]:
df.head()

In [ ]:
# group by year
# 함수가 전달되는 경우 함수의 파라미터로 index 값이 전달
year_group = df.groupby(lambda month : month.split('/')[2])

In [ ]:
year_group.groups

In [ ]:
year_group.size()

In [ ]:
# index의 경우에는 column name이 아닌, 레벨로 grouping 가능
month_group = df.groupby(level = 0)

In [ ]:
# 혹은 다음과 같이 index 자체를 명시 가능
month_group = df.groupby(level = 'Month')

In [ ]:
for name, group in month_group:
    print(name, '\n')
    print(group)

### multi index grouping

* index 항목 제거

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

* 2가지 컬럼으로 인덱싱

In [ ]:
# 멀티 레벨 인덱스를 생성
df = df.set_index(['Month', 'Stat'])
df.head()

* 멀티 인덱스에서 row 선택

In [ ]:
df.loc['08/01/2012']

In [ ]:
df.loc['08/01/2012'].loc['MatchesPlayed']

In [ ]:
df.loc['08/01/2012', 'MatchesPlayed']

* 멀티 인덱스의 레벨을 이용한 그룹핑

In [ ]:
# multi level grouping
month_stat_group = df.groupby(level = [0, 1])

In [ ]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)
    print()

In [ ]:
month_stat_group = df.groupby(level = ['Month', 'Stat'])

In [ ]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)

* row 선택하기

In [ ]:
df2 = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [ ]:
df2.head()

In [ ]:
df2.loc[0] # 0은 인덱스를 의미, 여기서는 단순한 순서

In [ ]:
df2 = df2.set_index('Month')
df2.head()

In [ ]:
# index가 Month로 바뀌었으므로
df2.loc['08/01/2012']

In [ ]:
# 0번째
df2.iloc[0]

In [ ]:
df2 = df2.reset_index()

* multiple index

In [ ]:
df2 = df2.set_index(['Month', 'Stat'])
df2.head()

In [ ]:
df2.iloc[0]

In [ ]:
# multi index의 경우 ','로 구별하여 인덱싱 가능
df2.loc['08/01/2012', 'MatchesPlayed']

* grouping with multi index

In [ ]:
df2.head()

In [ ]:
stat_group = df2.groupby(level = 'Stat')
for name, group in stat_group:
    print(name)
    print(group)

In [ ]:
# group의 sum method를 이용
# 각 리그당 경기 수와 골을 나타냄
stat_group.sum()

In [ ]:
# 혹은 level을 이용하여 dataframe level에서 계산 가능
df2.sum(level = 'Stat')

In [ ]:
total_goal_df = stat_group.sum()
total_goal_df.head()

###  각 리그의 경기당 평균골 구하기

* 평균 골 Series 생성

In [ ]:
goals_per_game_df = total_goal_df.loc['GoalsScored'] \
                        / total_goal_df.loc['MatchesPlayed']
goals_per_game_df

* dataframe으로 변경

In [ ]:
pd.DataFrame(goals_per_game_df)

* Transpose하여 row, column 변경

In [ ]:
goals_per_game_df = pd.DataFrame(goals_per_game_df).T
goals_per_game_df

* mapping dictionary로 인덱스 값 변경

In [ ]:
goals_per_game_df = goals_per_game_df \
                        .rename(index = \
                                {0 : 'goals per game'})
goals_per_game_df

* 새로운 row로 추가

In [ ]:
total_goal_df.append(goals_per_game_df)

* 소수점 2자리까지 출력하도록 설정

In [ ]:
pd.options.display.float_format='{:.2f}'.format
total_goal_df.append(goals_per_game_df)

### Formating
  - https://pyformat.info/ 참고

### aggregate method
 - groupby 객체에 aggregate(집합) 함수 적용하여 그룹별 데이터 확인 가능

In [ ]:
print(stat_group)
for name, group in stat_group:
    print(name)
    print(group)

In [ ]:
stat_group.sum()

In [ ]:
stat_group.aggregate(len)

In [ ]:
stat_group.aggregate(np.sum)

* multiple function 적용
  - 복수개의 column 레벨로 데이터 생성

In [ ]:
stat_group.aggregate([np.sum, np.mean, np.size])

### transform 함수

* 가상 sales data 실습
 - excel로 구성된 가상의 판매 데이터 이용하여 transformation 실습
 - excel 데이터를 읽기 위해서 xlrd 모듈이 필요
   - pip install xlrd

In [ ]:
df = pd.read_excel("data/sales_transactions.xlsx")

In [ ]:
df.head(13)

In [ ]:
df.groupby('order').sum()

In [ ]:
# ext price 컬럼만 sum
df.groupby('order')["ext price"].sum()

* 위의 정보를 다시 원래의 frame에 합치려면?
 - 아래와 같이 다시 merge 하여 합칠 수도 있음
 - 가능은 하지만, 더 쉽게 transform으로 가능

In [ ]:
order_total = df.groupby('order')["ext price"].sum().rename("total").reset_index()
order_total

In [ ]:
result = df.merge(order_total, on='order')
result["portion"] = result["ext price"] / result["total"]

In [ ]:
result

* **transform 함수 사용!**
 - group by 후 transform을 사용하면 원본의 row를 유지한 상태에서 transform에 전달된 함수를 호출하여 aggregation
 - row의 개수가 유지되므로 바로 cloumn 추가 방법을 사용하여 원본 dataframe에 추가 가능

In [ ]:
df.groupby('order')["ext price"].sum()

In [ ]:
df.groupby('order')["ext price"].transform(np.sum)

In [ ]:
df['total'] = df.groupby('order')["ext price"].transform(np.sum)
df["portion"] = df["ext price"] / df["total"]
df

###  pivot 
 - dataframe의 형태를 변경
 - 변경 시, index, column, data를 명시

* ** 식물실험 데이터**
 - ctrl, tr1, tr2 환경에서의 회차별 weight 관측 데이터

In [8]:
plant_df = pd.read_csv('data/PlantGrowth.csv')
plant_df.head(30)

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [9]:
plant_df
# control(ctrl)하여 실험한 값과, 두 개의 다른 조건(tr1, tr2)을 주었을 때의 실험에 대한 결과 
# data가 stacked 됨. (말 그대로 쌓여있음)
# 데이터 분석하기 편리하지 않음

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [ ]:
# 아래와 같이 filtering하여 각각의 결과를 뽑을 수 있으나, pivot, unstack을 이용하여 분석하기 용이하도록 변형 가능
plant_df[plant_df['group'] == 'ctrl']

In [ ]:
# dataframe reshaping
# 각각 index, columns, values로 명시하여 data frame의 shape을 변경 가능
plant_df.pivot(index = 'observation', 
               columns = 'group', 
               values = 'weight')

In [ ]:
# pivot_table 으로도 동일 결과 가능
pd.pivot_table(plant_df, values = 'weight', index = 'observation', columns='group')

In [ ]:
pd.pivot_table(plant_df, values = 'weight', columns='group')

In [ ]:
pd.pivot_table(plant_df, values = 'weight', columns='observation')

* **pivot, pivot_table의 차이**
 - pivot_table 
   - pivot 결과 생성 시, 값 선택이 vague한 경우, aggfunc 사용하여 채움


In [5]:
from collections import OrderedDict

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))
df = pd.DataFrame(table)

In [6]:
df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item1,Gold,3,3
3,Item1,Silver,4,4


In [ ]:
df.pivot(index='Item', columns='CType', values='USD')

In [ ]:
df.pivot(index='Item', columns='CType')

In [ ]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))

df = pd.DataFrame(table)

df

In [ ]:
# Item0과 Gold에 대해서 1, 3중 어떤 것을 채워야 할지 알 수 없음
df.pivot(index='Item', columns='CType', values='USD')

In [ ]:
df.pivot_table(index='Item', 
               columns='CType', 
               values='USD')

###  stack & unstack
 - stack : inner-most column -> inner-most index
 - unstack : inner-most index -> inner-most column
 
 - 둘은 역의 관계에 있음

In [10]:
plant_df

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [11]:
stacked = plant_df.set_index(['group', 'observation'])
stacked

weight
group observation        
ctrl  1              4.17
      2              5.58
      3              5.18
      4              6.11
      5              4.50
      6              4.61
      7              5.17
      8              4.53
      9              5.33
      10             5.14
trt1  1              4.81
      2              4.17
      3              4.41
      4              3.59
      5              5.87
      6              3.83
      7              6.03
      8              4.89
      9              4.32
      10             4.69
trt2  1              6.31
      2              5.12
      3              5.54
      4              5.50
      5              5.37
      6              5.29
      7              4.92
      8              6.15
      9              5.80
      10             5.26

In [12]:
# 가장 바깥 레벨의 row -> column으로 이동  level 기본 값이 -1 인덱스 마지막 값이 컬럼으로 올라감 0을 주면 group이 column이 됨
stacked.unstack()

weight                                                      
observation     1     2     3     4     5     6     7     8     9     10
group                                                                   
ctrl          4.17  5.58  5.18  6.11  4.50  4.61  5.17  4.53  5.33  5.14
trt1          4.81  4.17  4.41  3.59  5.87  3.83  6.03  4.89  4.32  4.69
trt2          6.31  5.12  5.54  5.50  5.37  5.29  4.92  6.15  5.80  5.26

In [14]:
# 레벨 0의 row -> column으로 이동
stacked.unstack(0)

# 레벨의 순서가 아닌, 네임으로도 명시 가능
# stacked.unstack(level = 'group')

weight            
group         ctrl  trt1  trt2
observation                   
1             4.17  4.81  6.31
2             5.58  4.17  5.12
3             5.18  4.41  5.54
4             6.11  3.59  5.50
5             4.50  5.87  5.37
6             4.61  3.83  5.29
7             5.17  6.03  4.92
8             4.53  4.89  6.15
9             5.33  4.32  5.80
10            5.14  4.69  5.26

In [15]:
# 컬럼을 다시 가장 하위의 row로 이동
stacked.unstack(0).stack()

weight
observation group        
1           ctrl     4.17
            trt1     4.81
            trt2     6.31
2           ctrl     5.58
            trt1     4.17
            trt2     5.12
3           ctrl     5.18
            trt1     4.41
            trt2     5.54
4           ctrl     6.11
            trt1     3.59
            trt2     5.50
5           ctrl     4.50
            trt1     5.87
            trt2     5.37
6           ctrl     4.61
            trt1     3.83
            trt2     5.29
7           ctrl     5.17
            trt1     6.03
            trt2     4.92
8           ctrl     4.53
            trt1     4.89
            trt2     6.15
9           ctrl     5.33
            trt1     4.32
            trt2     5.80
10          ctrl     5.14
            trt1     4.69
            trt2     5.26

### Concat

In [16]:
import pandas as pd
import numpy as np

In [17]:
df1 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [18]:
df1

,key1,value1
0,0,2.605446
1,1,-1.587288
2,2,1.022557
3,3,-0.079394
4,4,-0.598611
5,5,-0.368444
6,6,0.354386
7,7,-0.056556
8,8,0.088298
9,9,-0.063739


In [19]:
df2 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [20]:
df2

,key1,value1
0,0,-1.026712
1,1,-1.384530
2,2,-0.169667
3,3,-1.169527
4,4,-2.313941
5,5,-0.577217
6,6,-1.272959
7,7,-2.130147
8,8,0.368486
9,9,-0.020480


In [21]:
# axis=0 기본적으로 row를 따라 합침
pd.concat([df1, df2])

,key1,value1
0,0,2.605446
1,1,-1.587288
2,2,1.022557
3,3,-0.079394
4,4,-0.598611
5,5,-0.368444
6,6,0.354386
7,7,-0.056556
8,8,0.088298
9,9,-0.063739


In [22]:
# axis=1로 주어 column을 따라 합치는 것도 가능
pd.concat([df1, df2], axis=1)

,key1,value1,key1,value1
0,0,2.605446,0,-1.026712
1,1,-1.587288,1,-1.384530
2,2,1.022557,2,-0.169667
3,3,-0.079394,3,-1.169527
4,4,-0.598611,4,-2.313941
5,5,-0.368444,5,-0.577217
6,6,0.354386,6,-1.272959
7,7,-0.056556,7,-2.130147
8,8,0.088298,8,0.368486
9,9,-0.063739,9,-0.020480


In [23]:
# axis=0 기본적으로 row를 따라 합침
# 이때 기존의 인덱스를 무시하려면 ignore_index 파라미터를 True로 변경
pd.concat([df1, df2], ignore_index=True)

,key1,value1
0,0,2.605446
1,1,-1.587288
2,2,1.022557
3,3,-0.079394
4,4,-0.598611
5,5,-0.368444
6,6,0.354386
7,7,-0.056556
8,8,0.088298
9,9,-0.063739


* column 네임이 다를 경우

In [24]:
df3 = pd.DataFrame({'key2' : np.arange(10), 'value2' : np.random.randn(10)})

In [25]:
# 없는 컬럼에 대해서는 nan으로 채움
pd.concat([df1, df3], ignore_index=True)

,key1,key2,value1,value2
0,0.0,NaN,2.605446,NaN
1,1.0,NaN,-1.587288,NaN
2,2.0,NaN,1.022557,NaN
3,3.0,NaN,-0.079394,NaN
4,4.0,NaN,-0.598611,NaN
5,5.0,NaN,-0.368444,NaN
6,6.0,NaN,0.354386,NaN
7,7.0,NaN,-0.056556,NaN
8,8.0,NaN,0.088298,NaN
9,9.0,NaN,-0.063739,NaN


* index를 설정하고 concat

In [26]:
df11 = df1.set_index('key1')

In [27]:
df22 = df2.set_index('key1')

In [28]:
pd.concat([df11, df22])

,value1
key1,
0,2.605446
1,-1.587288
2,1.022557
3,-0.079394
4,-0.598611
5,-0.368444
6,0.354386
7,-0.056556
8,0.088298


In [29]:
result = pd.concat([df11, df22], axis=1)
result

,value1,value1
key1,,
0,2.605446,-1.026712
1,-1.587288,-1.384530
2,1.022557,-0.169667
3,-0.079394,-1.169527
4,-0.598611,-2.313941
5,-0.368444,-0.577217
6,0.354386,-1.272959
7,-0.056556,-2.130147
8,0.088298,0.368486


### Merge & Join

In [30]:
df1 = pd.DataFrame({'key1' : np.arange(8), 
                    'key2' : ['LA', 'LA', 'SF', 'SF', 'SF', 'NY', 'NY', 'LV'], 
                    'value1' : np.random.randn(8)})

In [31]:
df1

,key1,key2,value1
0,0,LA,-0.919033
1,1,LA,0.487031
2,2,SF,-0.044561
3,3,SF,0.779275
4,4,SF,-0.732686
5,5,NY,-1.125829
6,6,NY,1.397404
7,7,LV,-0.690789


In [32]:
df2 = pd.DataFrame({'key1' : [1, 1, 2, 2, 2, 3, 3], 
                    'key2' : ['LA', 'LA', 'KC', 'KC', 'NY', 'SF', 'SF'], 
                    'value1' : np.random.randn(7)})

In [33]:
df2

,key1,key2,value1
0,1,LA,0.703794
1,1,LA,0.540595
2,2,KC,-0.418359
3,2,KC,-0.169463
4,2,NY,0.899643
5,3,SF,-0.527145
6,3,SF,1.784105


* on
 - join 하고자 하는 대상이 되는 column 명시

In [34]:
pd.merge(df1, df2, on='key1')

,key1,key2_x,value1_x,key2_y,value1_y
0,1,LA,0.487031,LA,0.703794
1,1,LA,0.487031,LA,0.540595
2,2,SF,-0.044561,KC,-0.418359
3,2,SF,-0.044561,KC,-0.169463
4,2,SF,-0.044561,NY,0.899643
5,3,SF,0.779275,SF,-0.527145
6,3,SF,0.779275,SF,1.784105


In [35]:
pd.merge(df1, df2, on='key2')

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,-0.919033,1,0.703794
1,0,LA,-0.919033,1,0.540595
2,1,LA,0.487031,1,0.703794
3,1,LA,0.487031,1,0.540595
4,2,SF,-0.044561,3,-0.527145
5,2,SF,-0.044561,3,1.784105
6,3,SF,0.779275,3,-0.527145
7,3,SF,0.779275,3,1.784105
8,4,SF,-0.732686,3,-0.527145
9,4,SF,-0.732686,3,1.784105


* how parameter
 - SQL의 join과 같이 동작
 - inner : 기본 값, 일치 하는 값들로만 조인
 - left : left outer join과 같이 동작
 - right : right outer join과 같이 동작
 - outer : full outer join과 같이 동작, 즉 left + right

* left join
 - 왼쪽 df 기준으로 join, 없는 경우 nan 할당

In [36]:
pd.merge(df1, df2, on='key2', how='left')

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,-0.919033,1.0,0.703794
1,0,LA,-0.919033,1.0,0.540595
2,1,LA,0.487031,1.0,0.703794
3,1,LA,0.487031,1.0,0.540595
4,2,SF,-0.044561,3.0,-0.527145
5,2,SF,-0.044561,3.0,1.784105
6,3,SF,0.779275,3.0,-0.527145
7,3,SF,0.779275,3.0,1.784105
8,4,SF,-0.732686,3.0,-0.527145
9,4,SF,-0.732686,3.0,1.784105


* right join
 - 오른쪽 df 기준으로 join, 없는 경우 nan 할당

In [37]:
pd.merge(df1, df2, on='key2', how='right')

,key1_x,key2,value1_x,key1_y,value1_y
0,0.0,LA,-0.919033,1,0.703794
1,1.0,LA,0.487031,1,0.703794
2,0.0,LA,-0.919033,1,0.540595
3,1.0,LA,0.487031,1,0.540595
4,2.0,SF,-0.044561,3,-0.527145
5,3.0,SF,0.779275,3,-0.527145
6,4.0,SF,-0.732686,3,-0.527145
7,2.0,SF,-0.044561,3,1.784105
8,3.0,SF,0.779275,3,1.784105
9,4.0,SF,-0.732686,3,1.784105


* full join
 - left join 과 right join union

In [38]:
pd.merge(df1, df2, on='key2', how='outer')

,key1_x,key2,value1_x,key1_y,value1_y
0,0.0,LA,-0.919033,1.0,0.703794
1,0.0,LA,-0.919033,1.0,0.540595
2,1.0,LA,0.487031,1.0,0.703794
3,1.0,LA,0.487031,1.0,0.540595
4,2.0,SF,-0.044561,3.0,-0.527145
5,2.0,SF,-0.044561,3.0,1.784105
6,3.0,SF,0.779275,3.0,-0.527145
7,3.0,SF,0.779275,3.0,1.784105
8,4.0,SF,-0.732686,3.0,-0.527145
9,4.0,SF,-0.732686,3.0,1.784105


* **multi key 조인**
 - join 하고자 할 column 리스트로 구성

In [39]:
pd.merge(df1, df2, on=['key1', 'key2'])

,key1,key2,value1_x,value1_y
0,1,LA,0.487031,0.703794
1,1,LA,0.487031,0.540595
2,3,SF,0.779275,-0.527145
3,3,SF,0.779275,1.784105


In [40]:
pd.merge(df1, df2, on=['key1', 'key2'], how='left')

,key1,key2,value1_x,value1_y
0,0,LA,-0.919033,NaN
1,1,LA,0.487031,0.703794
2,1,LA,0.487031,0.540595
3,2,SF,-0.044561,NaN
4,3,SF,0.779275,-0.527145
5,3,SF,0.779275,1.784105
6,4,SF,-0.732686,NaN
7,5,NY,-1.125829,NaN
8,6,NY,1.397404,NaN
9,7,LV,-0.690789,NaN


In [41]:
# 나머지 column의 이름이 겹칠 경우 suffix 지정 가능
pd.merge(df1, df2, on=['key1', 'key2'], how='left', 
         suffixes=('_left', '_right'))

,key1,key2,value1_left,value1_right
0,0,LA,-0.919033,NaN
1,1,LA,0.487031,0.703794
2,1,LA,0.487031,0.540595
3,2,SF,-0.044561,NaN
4,3,SF,0.779275,-0.527145
5,3,SF,0.779275,1.784105
6,4,SF,-0.732686,NaN
7,5,NY,-1.125829,NaN
8,6,NY,1.397404,NaN
9,7,LV,-0.690789,NaN


* index가 있는 경우 merge
 - left_index : 왼쪽 프레임에서 인덱스를 사용할 경우 True 명시
 - right_index : 오른쪽 프레임에서 인덱스를 사용할 경우 True 명시

In [42]:
df11 = df1.set_index('key1')
df11

,key2,value1
key1,,
0,LA,-0.919033
1,LA,0.487031
2,SF,-0.044561
3,SF,0.779275
4,SF,-0.732686
5,NY,-1.125829
6,NY,1.397404
7,LV,-0.690789


In [43]:
df22 = df2.set_index('key1')
df22

,key2,value1
key1,,
1,LA,0.703794
1,LA,0.540595
2,KC,-0.418359
2,KC,-0.169463
2,NY,0.899643
3,SF,-0.527145
3,SF,1.784105


In [44]:
# index끼리 조인할 것이라고 명시 가능
pd.merge(df11, df22, left_index=True, right_index=True)

,key2_x,value1_x,key2_y,value1_y
key1,,,,
1,LA,0.487031,LA,0.703794
1,LA,0.487031,LA,0.540595
2,SF,-0.044561,KC,-0.418359
2,SF,-0.044561,KC,-0.169463
2,SF,-0.044561,NY,0.899643
3,SF,0.779275,SF,-0.527145
3,SF,0.779275,SF,1.784105
